In [1]:
from fastai.vision.all import *
from fastai.callback.wandb import WandbCallback
from sklearn.model_selection import StratifiedKFold

from ipyexperiments.ipyexperiments import IPyExperimentsPytorch
import timm,wandb,albumentations as A

import IPython.display as disp
from timm.models import safe_model_name, set_fast_norm
from timm.models.layers import create_classifier, SelectAdaptivePool2d

In [2]:
%env WANDB_SILENT=true
set_seed(42); set_fast_norm()
warnings.filterwarnings('ignore')

env: WANDB_SILENT=true


In [3]:
DATA_PATH = Path('/data/')
CSV_PATH = DATA_PATH/'train.csv'
IMAGE_FOLDER = DATA_PATH/'train'

LABELS = sorted(['broken', 'pure', 'discolored', 'silkcut'])
labels_class_map = {v:k for k,v in enumerate(LABELS)}
labels_class_map_rev = {v:k for k,v in labels_class_map.items()}

VIEWS = sorted(['top', 'bottom'])
views_class_map = {v:k for k,v in enumerate(VIEWS)}
views_class_map_rev = {v:k for k,v in views_class_map.items()}

TIMM_MODEL = "tf_efficientnet_b4_ns"
NFOLDS = 5; BS = 32; SZ = 320

TRAIN_AUG = A.Compose([
    A.Resize(SZ, SZ),
    A.Transpose(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.ShiftScaleRotate(rotate_limit=360, border_mode=0, p=0.75),
    A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1),
    A.Blur(p=0.5),
    A.CoarseDropout(max_height=int(32*(SZ/512)), max_width=int(32*(SZ/512)), p=0.75), 
])

CROP_PCT = 1.0
SCALE_SZ = int(math.floor(SZ / CROP_PCT))
VALID_AUG = A.Compose([A.Resize(SCALE_SZ, SCALE_SZ), A.CenterCrop(SZ, SZ)])

In [4]:
all_df = pd.read_csv(CSV_PATH)
all_df.head()

,seed_id,view,image,label
0,0,top,train/00000.png,broken
1,1,bottom,train/00001.png,pure
2,3,top,train/00003.png,broken
3,4,top,train/00004.png,pure
4,5,top,train/00005.png,discolored


In [5]:
folds = all_df.copy()
fold_splits = []

Fold = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=42)
for n, (train_index, val_index) in enumerate(Fold.split(folds, folds["label"])):
    fold_splits.append(val_index)
    folds.loc[val_index, 'fold'] = int(n)
folds['fold'] = folds['fold'].astype(int)

In [6]:
@Transform
def hflip(x: TensorTypes): return x.flip(3)

@Transform
def vflip(x: TensorTypes): return x.flip(2)

class AlbumentationsTransform(RandTransform):
    "A transform handler for multiple `Albumentation` transforms"
    split_idx,order=None,2
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)

In [7]:
from timm.models.helpers import group_modules, group_parameters

def timm_get_module_names(m):
    modules_names = group_modules(m, m.group_matcher(coarse=True))
    return L(modules_names.values())[:-1] #cut timm head

def timm_get_param_names(m):
    param_names = group_parameters(m, m.group_matcher(coarse=True))
    return L(param_names.values())[:-1] #cut timm head

def timm_body_params(timm_model, cut=-1):
    param_names = timm_get_param_names(timm_model)
    return L(param_names[:cut].concat(), param_names[cut:].concat())

def timm_split(m, cut=-1):
    "Splitter function for timm models"
    timm_model, head = m[0].model, m[1]
    body_groups = [g.map(timm_model.get_parameter) for g in timm_body_params(timm_model, cut)]
    return body_groups + [params(head)]

In [14]:
def get_dls(fold:int):
    
    aug_tfm = AlbumentationsTransform(TRAIN_AUG, VALID_AUG)
    item_tfms = [aug_tfm, ToTensor]
    batch_tfms = [IntToFloatTensor, Normalize.from_stats(*imagenet_stats)]
    
    dblock = DataBlock(
        blocks     = (ImageBlock, CategoryBlock(vocab=LABELS)),
        splitter   = IndexSplitter(fold_splits[fold]),
        get_x      = ColReader('image', pref=DATA_PATH),
        get_y      = [ColReader('label')],
        item_tfms  = item_tfms,
        batch_tfms = batch_tfms )
    
    dls = dblock.dataloaders(folds, bs=BS)
    return dls

def build_experiment(fold:int = 0, pretrained:bool = True):
    dls = get_dls(fold)
    
    learn = vision_learner(dls, TIMM_MODEL, n_out=dls.c, pretrained=True,
                           loss_func=CrossEntropyLossFlat(), metrics=[accuracy], 
                           splitter=partial(timm_split, cut=-3), act_layer=nn.Mish).to_fp16()
    return dls, learn

In [11]:
# dls = get_dls(0)
# dls.show_batch(max_n=6, unique=True)

In [12]:
# dls.valid.show_batch(max_n=6)

In [16]:
# with IPyExperimentsPytorch(exp_enable=False, cl_set_seed=42, cl_compact=True):      
#     dls, learn = build_experiment(0)
#     print(learn.lr_find(suggest_funcs=(valley, slide)))

In [21]:
VERSION  = "NB_EXP_023"
EXP_NAME = f"{VERSION}_efficientnet_b4" 

MODEL_FOLDER = Path(f'/runs/{VERSION}')
MODEL_FOLDER.mkdir(exist_ok=True)

n_epo = 15; base_lr = 0.003;
SUFFIX = f"{SZ}_bs{BS}_epo{n_epo}_mixup_ft"

print("EXP_NAME :", EXP_NAME, "; SUFFIX :", SUFFIX)
print("MODEL_FOLDER : ", MODEL_FOLDER)

EXP_NAME : NB_EXP_023_efficientnet_b4 ; SUFFIX : 320_bs32_epo15_mixup_ft
MODEL_FOLDER :  /runs/NB_EXP_023


In [22]:
for fold_idx in range(NFOLDS):
    with IPyExperimentsPytorch(exp_enable=False, cl_set_seed=42, cl_compact=True): 
        print()
        print('*' * 100)
        print(f'Training fold {fold_idx}')
        print('*' * 100)
        
        print(f'fold:{fold_idx}; model:{TIMM_MODEL}; sz:{SZ}; bs:{BS}; epochs:{n_epo}; base_lr:{base_lr}')
        
        dls, learn = build_experiment(fold_idx)
        learn.fine_tune(n_epo, base_lr, 
                        cbs=[SaveModelCallback(monitor="accuracy", comp=np.greater), MixUp(0.4)])

        learn = learn.load("model")
        learn = learn.to_fp32()

        dirpath = MODEL_FOLDER/f'{EXP_NAME}_{SUFFIX}_CV_{fold_idx:02d}'
        learn.save(dirpath, with_opt=False)


*** Experiment started with the Pytorch backend
Device: ID 0, NVIDIA GeForce RTX 3090 (24576 RAM)


****************************************************************************************************
Training fold 0
****************************************************************************************************
fold:0; model:tf_efficientnet_b4_ns sz:320; bs:32; epochs:15; base_lr:0.003


epoch,train_loss,valid_loss,accuracy,time
0,1.373429,1.120415,0.520070,00:52


Better model found at epoch 0 with accuracy value: 0.5200698375701904.


epoch,train_loss,valid_loss,accuracy,time
0,1.101282,0.853298,0.653054,01:13
1,0.950493,0.700573,0.721815,01:14
2,0.883481,0.613187,0.758115,01:14
3,0.854638,0.593146,0.772077,01:14
4,0.856081,0.684679,0.737173,01:14
5,0.805985,0.532784,0.786387,01:14
6,0.756990,0.560200,0.785689,01:14
7,0.763857,0.543735,0.791623,01:14
8,0.743270,0.548096,0.777661,01:14
9,0.712603,0.523279,0.788133,01:14


Better model found at epoch 0 with accuracy value: 0.6530541181564331.
Better model found at epoch 1 with accuracy value: 0.7218149900436401.
Better model found at epoch 2 with accuracy value: 0.7581151723861694.
Better model found at epoch 3 with accuracy value: 0.7720767855644226.
Better model found at epoch 5 with accuracy value: 0.7863874435424805.
Better model found at epoch 7 with accuracy value: 0.791623055934906.
Better model found at epoch 10 with accuracy value: 0.7993019223213196.
Better model found at epoch 11 with accuracy value: 0.8017451763153076.
Better model found at epoch 12 with accuracy value: 0.8038394451141357.
CPU: 139/9/3554 MB | GPU: 1318/6136/3780 MB | Time 0:19:29.966 | (Consumed/Peaked/Used Total)

*** Experiment started with the Pytorch backend
Device: ID 0, NVIDIA GeForce RTX 3090 (24576 RAM)


****************************************************************************************************
Training fold 1
***********************************************

...

Better model found at epoch 0 with accuracy value: 0.6396648287773132.
Better model found at epoch 1 with accuracy value: 0.6997206807136536.
Better model found at epoch 2 with accuracy value: 0.7496508359909058.
Better model found at epoch 3 with accuracy value: 0.7674580812454224.
Better model found at epoch 5 with accuracy value: 0.784217894077301.
Better model found at epoch 6 with accuracy value: 0.799580991268158.
Better model found at epoch 9 with accuracy value: 0.806913435459137.
CPU: 0/9/3572 MB | GPU: 72/5900/4224 MB | Time 0:19:35.650 | (Consumed/Peaked/Used Total)

*** Experiment started with the Pytorch backend
Device: ID 0, NVIDIA GeForce RTX 3090 (24576 RAM)


****************************************************************************************************
Training fold 4
****************************************************************************************************
fold:4; model:tf_efficientnet_b4_ns sz:320; bs:32; epochs:15; base_lr:0.003


epoch,train_loss,valid_loss,accuracy,time
0,1.373356,1.186397,0.474162,00:53


Better model found at epoch 0 with accuracy value: 0.4741620123386383.


epoch,train_loss,valid_loss,accuracy,time
0,1.094462,0.885479,0.637570,01:14
1,0.957354,0.684039,0.732193,01:14
2,0.893117,0.615042,0.763617,01:14
3,0.835585,0.550042,0.779330,01:14
4,0.833418,0.593543,0.767458,01:14
5,0.800637,0.558039,0.781774,01:14
6,0.771048,0.526447,0.791899,01:14
7,0.770675,0.515084,0.802025,01:14
8,0.755155,0.506469,0.797835,01:14
9,0.724310,0.480620,0.810056,01:14


Better model found at epoch 0 with accuracy value: 0.6375698447227478.
Better model found at epoch 1 with accuracy value: 0.7321927547454834.
Better model found at epoch 2 with accuracy value: 0.7636173367500305.
Better model found at epoch 3 with accuracy value: 0.7793295979499817.
Better model found at epoch 5 with accuracy value: 0.7817737460136414.
Better model found at epoch 6 with accuracy value: 0.7918994426727295.
Better model found at epoch 7 with accuracy value: 0.8020251393318176.
Better model found at epoch 9 with accuracy value: 0.8100558519363403.
Better model found at epoch 12 with accuracy value: 0.8131983280181885.
CPU: 0/9/3572 MB | GPU: -40/5902/4184 MB | Time 0:19:34.759 | (Consumed/Peaked/Used Total)


In [24]:
pattern = f'{EXP_NAME}_{SUFFIX}_CV_' + '{fold:02d}'
folds_val_score = []

with IPyExperimentsPytorch(exp_enable=False, cl_set_seed=42, cl_compact=True):
    
    for fold_index in range(NFOLDS):
        
        dls, learn = build_experiment(fold_index, pretrained=False)
        learn.model_dir = Path(MODEL_FOLDER)
        
        checkpoint = pattern.format(fold=fold_index)
        print(f"==> Load checkpoint from : {checkpoint!r} ({fold_index:02d})")
        
        learn.load(checkpoint)
        
        # no augs
        tst_dl = dls.valid.new()
        
        # hflip augs
        tst_dl_hflip = dls.valid.new()
        tst_dl_hflip.after_batch.add(hflip)
        
        # vflip augs
        tst_dl_vflip = dls.valid.new()
        tst_dl_vflip.after_batch.add(vflip)
        
        # hflip + vflip augs
#         tst_dl_hlip_vflip = dls.valid.new()
#         tst_dl_hlip_vflip.after_batch.add([hflip,vflip])
            
#         dls = [tst_dl,tst_dl_hflip,tst_dl_vflip,tst_dl_hlip_vflip]
        dls = [tst_dl,tst_dl_hflip,tst_dl_vflip]
        
        n = len(dls)
        learn([event.before_fit, event.before_epoch])
        
        aug_preds = []
        with learn.no_mbar():
            if hasattr(learn,'progress'): learn.progress.mbar = master_bar(list(range(n)))
            
            for i in learn.progress.mbar if hasattr(learn,'progress') else range(n):
                learn.epoch = i
                with dls[i].dataset.set_split_idx(1): preds,targs = learn.get_preds(dl=dls[i], inner=True)
                aug_preds.append(preds[None])
        
        learn.epoch = n
        
        aug_preds = torch.cat(aug_preds)
        aug_preds = aug_preds.mean(0)
        acc = accuracy(aug_preds, targs).item()
        print(f"* Accuracy : {acc}")
        print()
        
        folds_val_score.append(acc)
        learn(event.after_fit)
        
           
print()
print('*' * 100)
print(f'Run summary: ')
print('*' * 100)
print("folds score: ", [np.round(s, 5) for s in folds_val_score])
print("Avg: {:.5f}".format(np.mean(folds_val_score)))
print("Std: {:.5f}".format(np.std(folds_val_score)))


*** Experiment started with the Pytorch backend
Device: ID 0, NVIDIA GeForce RTX 3090 (24576 RAM)

==> Load checkpoint from : 'NB_EXP_023_efficientnet_b4_320_bs32_epo15_mixup_ft_CV_00' (00)


* Accuracy : 0.803490400314331

==> Load checkpoint from : 'NB_EXP_023_efficientnet_b4_320_bs32_epo15_mixup_ft_CV_01' (01)


* Accuracy : 0.8111692667007446

==> Load checkpoint from : 'NB_EXP_023_efficientnet_b4_320_bs32_epo15_mixup_ft_CV_02' (02)


* Accuracy : 0.8163408041000366

==> Load checkpoint from : 'NB_EXP_023_efficientnet_b4_320_bs32_epo15_mixup_ft_CV_03' (03)


* Accuracy : 0.8145949840545654

==> Load checkpoint from : 'NB_EXP_023_efficientnet_b4_320_bs32_epo15_mixup_ft_CV_04' (04)


* Accuracy : 0.81948322057724

CPU: 39/14/3643 MB | GPU: 334/992/2744 MB | Time 0:01:11.233 | (Consumed/Peaked/Used Total)

****************************************************************************************************
Run summary: 
****************************************************************************************************
folds score:  [0.80349, 0.81117, 0.81634, 0.81459, 0.81948]
Avg: 0.81302
Std: 0.00547
